In [1]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import os, sys

In [19]:
def preprocess_image(image):
    try:
        ori = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(ori, cv2.COLOR_BGR2GRAY)

        gradX = cv2.Sobel(gray, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
        gradY = cv2.Sobel(gray, ddepth=cv2.CV_32F, dx=0, dy=1, ksize=-1)

        # subtract the y-gradient from the x-gradient
        gradient = cv2.subtract(gradX, gradY)
        gradient = cv2.convertScaleAbs(gradient)

        # blur and the image
        blurred = cv2.blur(gradient, (9, 9))

        # threshold the image
        (_, thresh) = cv2.threshold(blurred, 90, 255, cv2.THRESH_BINARY)

        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 25))
        closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

        # perform a series of erosions and dilations
        closed = cv2.erode(closed, None, iterations=4)
        closed = cv2.dilate(closed, None, iterations=13)

        # find Contour
        (cnts,_) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        c = sorted(cnts, key=cv2.contourArea, reverse=True)[0]

        # compute the rotated bounding box of the largest contour
        rect = cv2.minAreaRect(c)
        box = np.int0(cv2.boxPoints(rect))

        # find image bounding box
        Xs = [i[0] for i in box]
        Ys = [i[1] for i in box]

        x1 = min(Xs)
        y1 = min(Ys)     
        x2 = max(Xs)
        y2 = max(Ys)

        if x1 < 0:
            x1 = 0
        if y1 < 0:    
            y1 = 0
        
        hight = y2 - y1
        width = x2 - x1
        crop_img = image[y1:y1+hight, x1:x1+width]

        img_result = tf.image.resize_with_crop_or_pad(crop_img,200,200)

        return img_result
    except IndexError:
        print("list out of range")
        return -1

In [20]:
if __name__ == "__main__":
    
    # open folder
    path = r"./images/"
    dirs = os.listdir(path)

    # input images
    for file in dirs:
        if file != ".ipynb_checkpoints":
            path = r"./images/%s"%(file)
            image = cv2.imread(path)

            if image is None:
                print ('Error opening image: ' + path)
            
            print(file)
            #transfer image type EagerTensor to numpy.ndarray
            img_result = preprocess_image(image)
            img_result = np.array(img_result)
            
            # save image
            cv2.imwrite("./images_out/%s"%(file), img_result)
            plt.show()
            


089037461.jpg
089037462.jpg
089037463.jpg
089037464.jpg
089037465.jpg
089037466.jpg
089037467.jpg
089037468.jpg
089037469.jpg
089037470.jpg
089037471.jpg
089037472.jpg
089037473.jpg
089037474.jpg
089037475.jpg
089037476.jpg
089037477.jpg
089037478.jpg
089037479.jpg
089037480.jpg
089037481.jpg
089037482.jpg
089037483.jpg
089037484.jpg
089037485.jpg
089037486.jpg
089037487.jpg
089037488.jpg
089037489.jpg
089037490.jpg
089037491.jpg
089037492.jpg
089037493.jpg
089037494.jpg
089037495.jpg
089037496.jpg
089037497.jpg
089037498.jpg
089037499.jpg
089037500.jpg
089037501.jpg
089037502.jpg
089037503.jpg
089037504.jpg
089037505.jpg
089037506.jpg
089037507.jpg
089037508.jpg
089037509.jpg
089037510.jpg
089037511.jpg
089037512.jpg
089037513.jpg
089037514.jpg
089037515.jpg
089037516.jpg
089037517.jpg
089037518.jpg
089037519.jpg
089037520.jpg
089037521.jpg
089037522.jpg
089037523.jpg
089037524.jpg
089037525.jpg
089037526.jpg
089037527.jpg
089037528.jpg
089037529.jpg
089037530.jpg
089037531.jpg
089037